In [37]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import ee
import geemap
import folium
import smile

In [38]:
# ee.Authenticate()

In [39]:
ee.Initialize()

In [40]:
# This function defines a method to add an Earth Engine layer and is nessisary for Folium to work

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [41]:
# Import Image from Earth Engine account

north_image = ee.Image('projects/ee-arron/assets/Orthomosaic')

In [42]:
# Visualisation Perameters

vis_rgb = {
    'bands': ['b3', 'b2', 'b1'], 
    'min': 0, 
    'max': [0.2081535905599594,0.19500049948692322,0.1458057314157486],        # Max values are the max band values for each respective band taken from the image meta-data
    'gamma': 2
}

vis_531 = {
    'bands': ['b5', 'b3', 'b1'], 
    'min': 0, 
    'max': [0.39981067180633545, 0.2081535905599594, 0.2081535905599594,],      # Max values are the max band values for each respective band taken from the image meta-data
    'gamma': 2
}

vis_ndvi = {
    'palette': ['FFF8DC', '006400'],
    'min': 0,
    'max': 1,
}

vis_ndre = {
    'palette': ['FFF8DC', '006400'],
    'min': 0,
    'max': 0.4,
}

In [43]:
# North RGB Visualisation

north_map = folium.Map(location = ['16.538', '120.662'], zoom_start = 16.5)      # Create a new folium map called rgb
north_map.add_ee_layer(north_image, vis_rgb, 'North RGB')                        # Add new layer for north image with rgb vis perams

display(north_map)

In [44]:
# North 531 Visualisation

north_map = folium.Map(location = ['16.538', '120.662'], zoom_start = 16.5)
north_map.add_ee_layer(north_image, vis_531, 'North 531')

display(north_map)

In [45]:
# North NDVI Visualisation

north_ndvi = north_image.normalizedDifference(['b5','b3'])           # Calculate NDVI using b5 (NIR) and b3 (Red)

north_map = folium.Map(location = ['16.538', '120.662'], zoom_start = 16.5)
north_map.add_ee_layer(north_ndvi, vis_ndvi, 'North NDVI')
display(north_map)

In [46]:
# North NDRE Visualisation

north_ndre = north_image.normalizedDifference(['b5','b4'])           # Calculate NDRE using b5 (NIR) and b4 (Red Edge)

north_map = folium.Map(location = ['16.538', '120.662'], zoom_start = 16.5)
north_map.add_ee_layer(north_ndre, vis_ndre, 'North NDRE')
display(north_map)

In [47]:
# Unsupervised Clustering Classifier

Map = geemap.Map()                                          # Create an Empty Map
Point = ee.Geometry.Point([16.538, 120.662])                # Create point to center view on north raster
Map.centerObject(Point, 16)
Map.addLayer(north_image, vis_rgb, "Geemap RGB")            # Add RGB Visualisation to GEEMap using 'vis_rgb' perameters

In [48]:
# Create Training Data

unsupervised_training = north_image.sample(                 # Randomly sample pixel values in 'north_image'
    **{
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,
    }
)

Map.addLayer(unsupervised_training, {}, 'training', False)  # Add these training points to map to visualise
# Map

In [49]:
# Train Classifier

n_clusters = 6
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(unsupervised_training)      # Train Clustering Classifier

In [50]:
# Classify Image

result = north_image.cluster(clusterer)                     # Create classified image 'result'

Map.addLayer(result.randomVisualizer(), {}, 'clusters')     # Add image to map with random colours representing classes
Map

Map(center=[59.338, -163.46200000000002], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [51]:
# Get Info on Image

props = geemap.image_props(north_image)
props.getInfo()

{'NOMINAL_SCALE': 0.05950399817906165,
 'system:asset_size': '1936.58175 MB',
 'system:band_names': ['b1', 'b2', 'b3', 'b4', 'b5', 'b6'],
 'system:id': 'projects/ee-arron/assets/Orthomosaic',
 'system:version': 1672747282055360.0}

In [52]:
# Export Classified Image

# task = ee.batch.Export.image.toDrive(             # Define a task to export image to google drive
#   image=result,
#   description='unsupervised_classification_result',
#   folder='GEE_Exports',
#   fileNamePrefix='unsupervised_classification_result',
#   scale=30,
#   region=north_image.geometry().bounds().getInfo()['coordinates'][0],
#   crs='EPSG:3857'
# )

# task.start()                                      # Start Export

In [53]:
# Supervised Classifier

# Current attempt to train a supervised classifier off training data
# Work in progress, ive left it in incase its useful but it might be easier to start from scratch using the training data



bands = ['b1', 'b2', 'b3', 'b4', 'b5']                                     # List of band names in the training data

north_selected = north_image.select(bands)

training_data = ee.FeatureCollection('projects/ee-arron/assets/Training')  # Load Training Data .csv and convert to ee.FC
# training_data = pd.read_csv("/Users/Arron/Desktop/TrainingData.csv")

training_fraction = 0.7                                                    # Create Train/Test Split
train = training_data.filter(ee.Filter.lt('random', training_fraction))
test = training_data.filter(ee.Filter.gte('random', training_fraction))
# training_data['random'] = np.random.rand(len(training_data))
# training_fraction = 0.7
# train = training_data[training_data['random'] < training_fraction]
# validate = training_data[training_data['random'] >= training_fraction]

classifier = ee.Classifier.libsvm().train(train, 'Class', bands)           # Train SVM Classifier

classified_image = north_selected.classify(classifier)                     # Classify Image

Map.addLayer(classified_image, {'min': 0, 'max': 1, 'palette': ['red', 'green', 'blue']})    # Visualise the result

EEException: No valid training data were found.

In [ ]:
import json

# Open the GeoJSON file
with open('/Users/Arron/Desktop/Philippines_Fieldwork/TrainingData3.geojson') as f:
    geojson = json.load(f)


# Create an empty FeatureCollection
TrainingDataFC = ee.FeatureCollection([])

# Loop through the features in the GeoJSON file
for feature in geojson['features']:
    # Get the geometry and properties
    geometry = ee.Geometry.MultiPolygon(feature['geometry']['coordinates'])
    properties = feature['properties']

    # Create a new Feature from the GeoJSON feature
    ee_feature = ee.Feature(geometry, properties)

    # Add the Feature to the FeatureCollection
    TrainingDataFC = TrainingDataFC.merge(ee_feature)

# Print the FeatureCollection
print(TrainingDataFC.getInfo())


In [ ]:
image = ee.Image('projects/ee-arron/assets/Orthomosaic')

image = image.clip(TrainingDataFC)


RGB_vis = {
    'bands': ['b3', 'b2', 'b1'], 
    'min': 0, 
    'max': [0.2081535905599594,0.19500049948692322,0.1458057314157486],      # Max values are the max band values for each respective band taken from the image meta-data
    'gamma': 2
};

clip_map = folium.Map(location = ['16.538', '120.662'], zoom_start = 16.5)
clip_map.add_ee_layer(image, RGB_vis, 'L5Image')

display(rgb_map)


In [ ]:
image = ee.Image('projects/ee-arron/assets/Orthomosaic')


# mapping function to generate random points within polygons
def generate_points(feature):
    return ee.Feature(feature.geometry().randomPoints(n))

# generate random points within polygons
random_points_fc = TrainingDataFC.map(generate_points)

# extract pixel values from the image at the random points
point_values = image.reduceRegions(collection=random_points_fc,reducer=ee.Reducer.first())

# joining the polygon class values with the pixel values
dataset = random_points_fc.map(lambda feature: feature.set(point_values.get(feature.id()).set("class", feature.get("class"))))

In [ ]:
Map = geemap.Map()

# Load the image
image = ee.Image('projects/ee-arron/assets/Orthomosaic')

# Load the feature collection
trainingData = ee.FeatureCollection('projects/ee-arron/assets/TrainingData')

# Extract the values of the image pixels within the boundaries of the features


bands = ['b1', 'b2', 'b3']

# Extract the pixel values from the image
image_values = image.select(bands).sampleRegions(collection=trainingData, properties=['class'], scale=10)

# Print the sample dataset
print(image_values.getInfo())

# Print the samples
# print(trainingData.getInfo())




# # Split the extracted values into two groups: one for training and one for testing
# randomValues = extractedValues.randomColumn()
# # Split the collection into two groups using a "lt" filter
# trainingPartition = extractedValues.filter(ee.Filter.lt('random', 0.5))
# testingPartition = extractedValues.filter(ee.Filter.lt('random', 0.5))






# # Extract the variables to use for training
# variables = ['variable1', 'variable2', 'variable3']

# # Create the classifier
# classifier = ee.Classifier.randomForest(numberOfTrees=100)

# # Train the classifier
# trainedClassifier = classifier.train(trainingPartition, 'class', variables)

# # Test the classifier
# testingAccuracy = trainedClassifier.confusionMatrix(testingPartition)

# # Print the testing accuracy
# print('Testing accuracy:', testingAccuracy)

# # Add the training data to the map
# Map.addLayer(trainingData, {}, 'training data')

# # Display the map
# Map.show()



# vis_params = {
#     'color': '000000',
#     'colorOpacity': 1,
#     'pointSize': 3,
#     'pointShape': 'circle',
#     'width': 2,
#     'lineType': 'solid',
#     'fillColorOpacity': 0.66,
# }

# palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']

# Map.add_layer(trainingData, {'color': 'FF0000'})

# Map.centerObject(North_Test, 16)

# Map

In [ ]:
properties = trainingData.first().propertyNames().getInfo()
print(properties)